table of contents  
1. Prepare  
2. Data Read, Preprocess, EDA  
3. Aggregate, Visualize

# Prepare

## Import Library

In [1]:
import os, time, sys, importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Define Static Parameter

In [2]:
base_dir = os.path.join('..')
data_dir = os.path.join(base_dir, 'datasets', 'positional_data')

sys.path.append(os.path.join(base_dir,'..','LaurieOnTracking','src'))
import Metrica_IO as mio
import Metrica_Viz as mviz
import Metrica_Velocities as mvel

# Data Read Preprocess, EDA

## Read Dataset

In [3]:
os.listdir(data_dir)

['LIV_[2]-1_CHE.csv', 'liverpool_2019.csv', 'Real_vs_Barcelona_Calma.csv']

In [4]:
df = pd.read_csv(os.path.join(data_dir, 'liverpool_2019.csv'), index_col=['play', 'frame'])

In [5]:
df.head()

Unnamed: 0 bgcolor        dx        dy  \
play                          frame                                           
Liverpool [3] - 0 Bournemouth 0               0     NaN  0.000000  0.000000   
                              1               1     NaN  0.185745  1.217580   
                              2               2     NaN  0.178659  1.171133   
                              3               3     NaN  0.171573  1.124685   
                              4               4     NaN  0.164488  1.078238   

                                    edgecolor  player  player_num team  \
play                          frame                                      
Liverpool [3] - 0 Bournemouth 0           NaN       0         NaN  NaN   
                              1           NaN       0         NaN  NaN   
                              2           NaN       0         NaN  NaN   
                              3           NaN       0         NaN  NaN   
                              4           NaN       0         NaN  NaN   

                                             x          y    z  
play                          frame                             
Liverpool [3] - 0 Bournemouth 0      46.394558  11.134454  0.0  
                              1      46.580302  12.352034  0.0  
                              2      46.758961  13.523166  0.0  
                              3      46.930535  14.647852  0.0  
                              4      47.095022  15.726090  0.0

In [6]:
df.index.get_level_values('play').unique()

Index(['Liverpool [3] - 0 Bournemouth', 'Bayern 0 - [1] Liverpool',
       'Fulham 0 - [1] Liverpool', 'Southampton 1 - [2] Liverpool',
       'Liverpool [2] - 0 Porto', 'Porto 0 - [2] Liverpool',
       'Liverpool [4] - 0 Barcelona', 'Liverpool [1] - 0 Wolves',
       'Liverpool [3] - 0 Norwich', 'Liverpool [2] - 1 Chelsea',
       'Liverpool [2] - 1 Newcastle', 'Liverpool [2] - 0 Salzburg',
       'Genk 0 - [3] Liverpool', 'Liverpool [2] - 0 Man City',
       'Liverpool [1] - 0 Everton', 'Liverpool [2] - 0 Everton',
       'Bournemouth 0 - 3 Liverpool', 'Liverpool [1] - 0 Watford',
       'Leicester 0 - [3] Liverpool'],
      dtype='object', name='play')

## Viz All

In [8]:
x_size, y_size = 106, 68

def pivot_table_for_Metrica(df):
    df = df.pivot_table(index='frame', columns='player', values=['x', 'y'])
    df.columns = [f'ball_{c_tuple[0]}' if c_tuple[1] == 0 else f'{c_tuple[1]}_{c_tuple[0]}' for c_tuple in df.columns]

    c_x_list, c_y_list = [c for c in df.columns if c.endswith('_x')], [c for c in df.columns if c.endswith('_y')]
    
    df[c_x_list] = df[c_x_list].applymap(lambda x: (x_size/2)*((x-50)/50))
    df[c_y_list] = df[c_y_list].applymap(lambda x: (y_size/2)*((x-50)/50))

    return df

In [9]:
for play in df.index.get_level_values('play').unique():
    df_tmp = df.loc[play]
    bgcolor_list = df_tmp.bgcolor.dropna().unique().tolist()
    
    # split, preprocess
    df_list = [pivot_table_for_Metrica(df_tmp[df_tmp.bgcolor.isin([bgcolor, np.nan])]) for bgcolor in bgcolor_list]
    
    mviz.save_match_clip(df_list[0], df_list[1], fpath=os.path.join(base_dir, 'reports', 'movie'), fname=play, figax=mviz.plot_pitch(field_color='twitter_dark_mode'), team_colors=bgcolor_list, title=play)

Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
